In [1]:
using Pkg
Pkg.activate("."); 
Pkg.instantiate()

  Activating environment at `C:\Users\andi\Documents\Uni\Köln\21WS\ATIS3\GroupProject\ATIES3_group_project-main\Project.toml`


In [2]:
using JuMP, Plots, CPLEX, DataFrames, XLSX, IterTools, Statistics, StatsPlots

In [3]:
data = Dict()
data["time_series"] = DataFrame(XLSX.readtable("data_ATIS3.xlsx", "time_series")...)
data["scenarios"] = DataFrame(XLSX.readtable("data_ATIS3.xlsx", "scenarios")...) ;

Orientiert an C1_Producer_Pool.

\Omega = 100 Szenarien für jede betrachtete Periode t Element 1:T

Ich habe die Parameter jetzt nicht in einem Dict gespeichert, weil das das Aufrufen später weniger voluminös macht.

In [35]:
### Sets ### (or rather, set cardinalities)
# Number scenarios
Ω = 100
# Number time periods
T = 1
# Initial and final period
ti = 4
tf = ti + T-1

### Scalars ###
# Factors to FC that limit DA bids
FC_DA_fac_lo = 0.85
FC_DA_fac_up = 1.15
# Risk
β = 0.0
α = 0.95
# Scaling factor FC
q_FC_scal = 0.025

### Vectors ###
## Length T
# Prices in DA, 15 and ID
p_DA  = data["time_series"][ti:tf,"p_da"]
p_15  = data["time_series"][ti:tf,"p_15"]
p_ID3 = data["time_series"][ti:tf,"p_id3"]
# Power ForeCasted and real
q_FC = data["time_series"].fc_wind_bw[ti:tf] .* q_FC_scal
# Forecast type
    # The value of a forecast q_FC leads to different distributions of its error Δq_FC.
    # We distinguish between low (<250 MW), mid (250 MW < Δq_FC < 1250 MW) and high (> 1250 MW).
    # Depending on the forecast q_FC[t] in a period t, we save the forecast type as 1=low,
    # 2=mid and 3=high. We expect most entries to be 2=mid.
    # Later on, F_type[t] is used to select the right column of the 
    # forecast error distribution matrix Δq_FC. Concrete: Δq_FC[ω, F_type[t]].
    # This way, we only have to save a vector with length T and a matrix with
    # dim ω x 3, instead of a matrix with dim ω x T with many repeating entries.
F_type = fill(2, T)
for t in 1:T
    if q_FC[t] <= 250 * q_FC_scal
        F_type[t] = 1
    elseif q_FC[t] >= 1250 * q_FC_scal
        F_type[t] = 3
    end
end
## Length Ω
# FC error sign; 1st stage decission; # of pos and neg scenarios is expected to be the same for low, mid and high
F_sgn = vcat(-1 * ones(size(data["scenarios"].neg)), ones(size(data["scenarios"].pos)))
# Relative FC errors
Δq_FC = vcat(data["scenarios"].neg, data["scenarios"].pos)
# Non-anticipativity vector (same 1st stage -> same p_15)
A = ones(length(F_sgn)-1)
for a in 1:length(F_sgn)-1
    if F_sgn[a] != F_sgn[a+1]
        A[a] = 0
    end
end
# Scenario probablilities
pi = 1/Ω * ones(Ω)


### Matrices ###
# Forecast error matrix (Ω x 3)
    # Relative error of the forecast used for each scenario ω ϵ Ω,
    # depending on the forecast type F_type being low, mid or high (see text above)
Δq_FC = hcat(
    vcat(data["scenarios"].neg_u250, data["scenarios"].pos_u250),
    vcat(data["scenarios"].neg, data["scenarios"].pos),
    vcat(data["scenarios"].neg_o1250, data["scenarios"].pos_o1250))
;

In [5]:
wind = Model(CPLEX.Optimizer);

In [6]:
# Power
q_DA = @variable(wind, [1:T]) # No ω dependence due to non-anticipativity!?
q_15 = @variable(wind, [1:T, 1:Ω])
# q_ID is in fact not needed, as it only depends on q_DA, q_15 and q_r=q_FC[t]*(1+Δq_FC[ω])

# Risk
η = @variable(wind, [1:T, 1:Ω])
ζ = @variable(wind, [1:T]);

In [7]:
# Realized power
q_r = @expression(wind, [t in 1:T, ω in 1:Ω],
    q_FC[t] * (1 + Δq_FC[ω,F_type[t]]))
# Revenue
R = @expression(wind, [t in 1:T, ω in 1:Ω],
    p_DA[t] * q_DA[t]
    + p_15[t] * q_15[t,ω]
    + p_ID3[t] * (q_r[t,ω] - q_DA[t] - q_15[t, ω]))
# Risk
CVaR = @expression(wind, [t in 1:T],
    (ζ[t] - 1/(1-α) * sum(pi[ω]*η[t,ω] for ω in 1:Ω)));

In [8]:
@objective(wind, Max,
    sum(sum(pi[ω]*R[t,ω] for ω in 1:Ω) + β * CVaR[t] for t in 1:T));

In [9]:
# DA bid within 85% and 115% of forecast
MinMaxPowerDA = @constraint(wind, [t in 1:T],
    FC_DA_fac_lo*q_FC[t] <= q_DA[t] <= FC_DA_fac_up*q_FC[t]);
# Limits for total sold power; max is given by realization of last scenario of each period
MaxPowerTot = @constraint(wind, [t in 1:T, ω in 1:Ω],
    0 <= q_DA[t]+q_15[t, ω] <= q_r[t,end]); 
# Non-anticipativity; gets only activated, if 1st stage of two scenarios is equal;
# q_15 has to be the same then.
nonAnticip = @constraint(wind, [t in 1:T, ω in 1:Ω-1],
    A[ω] * (q_15[t,ω]-q_15[t,ω+1]) == 0)
# Risk
CVaRConst = @constraint(wind, [t in 1:T, ω in 1:Ω],
    ζ[t] - R[t,ω] <= η[t,ω])
ηLimLow = @constraint(wind, [t in 1:T, ω in 1:Ω],
    η[t,ω] >= 0);

In [10]:
optimize!(wind)
termination_status(wind)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Parallel mode: deterministic, using up to 4 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 3 threads...
Tried aggregator 1 time.
LP Presolve eliminated 2010 rows and 1010 columns.
Aggregator did 990 substitutions.
Reduced LP has 1000 rows, 1030 columns, and 3000 nonzeros.
Presolve time = 0.01 sec. (3.06 ticks)
Symmetry aggregator did 1990 additional substitutions.
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =            52.910000

Dual crossover.
  Dual:  Fixing 196 variables.
      195 DMoves:  Infeasibility  0.00000000e+00  Objective  5.24329058e+03
        0 DMoves:  Infeasibility  0.00000000e+00  Objective  5.24329058e+03
  Dual:  Pushed 196, exchanged 0.
  Primal:  Fixed no variables.

Dual simplex solved model.



OPTIMAL::TerminationStatusCode = 1

In [11]:
value.(R)

10×100 Matrix{Float64}:
 128.253   156.828   174.517   191.3     …  1401.8    1681.91  2365.74
 161.909   189.933   207.281   223.739      1410.89   1685.59  2356.23
 126.636   155.6     173.53    190.541      1417.53   1701.45  2394.59
 140.069   168.281   185.746   202.315      1397.43   1673.98  2349.12
 172.109   197.97    213.979   229.167      1324.68   1578.19  2197.06
  99.6208  125.389   141.341   156.475   …  1248.07   1500.66  2117.31
  86.793   113.84    130.584   146.469      1292.25   1557.38  2204.65
  63.7364   87.0627  101.503   115.202      1103.35   1332.01  1890.23
  91.5924  114.946   129.404   143.12       1132.44   1361.37  1920.26
  50.9645   66.8069   76.6142   85.9184      757.034   912.33  1291.45

## Model as Function

In [36]:
function wind_opt(β=0.0)
    wind = Model(CPLEX.Optimizer)
    set_silent(wind)

    @variables(wind, begin
        # Powers
        q_DA[1:T] >= 0
        q_15[1:T,1:Ω]
        # Risk
        η[1:T,1:Ω] >= 0
        ζ[1:T]
    end)
    
    @expression(wind, q_r[t in 1:T, ω in 1:Ω], q_FC[t] * (1 + Δq_FC[ω,F_type[t]])) # Realized power
    @expression(wind, R[t in 1:T, ω in 1:Ω], # Revenue
        p_DA[t] * q_DA[t]
        + p_15[t] * q_15[t,ω]
        + p_ID3[t] * (q_r[t,ω] - q_DA[t] - q_15[t, ω]))
    @expression(wind, CVaR[t in 1:T], (ζ[t] - 1/(1-α) * sum(pi[ω]*η[t,ω] for ω in 1:Ω))) # Risk
    
    @objective(wind, Max, sum(sum(pi[ω]*R[t,ω] for ω in 1:Ω) + β * CVaR[t] for t in 1:T)) #
    println("β: ", β)
    
    @constraints(wind, begin
        MinMaxPowerDA[t in 1:T], FC_DA_fac_lo*q_FC[t] <= q_DA[t] <= FC_DA_fac_up*q_FC[t]
        MaxPowerTot[t in 1:T, ω in 1:Ω], 0 <= q_DA[t]+q_15[t, ω] <= q_r[t,end]
        NonAnticip15[t in 1:T, ω in 1:Ω-1], A[ω] * (q_15[t,ω]-q_15[t,ω+1]) == 0
        CVaRconstr[t in 1:T, ω in 1:Ω], ζ[t] - R[t,ω] <= η[t,ω]
    end)
    
    optimize!(wind)
    
    println("p_DA ", p_DA[1])
    println("p_15 ", p_15[1])
    println("p_ID3 ", p_ID3[1])
    println("q_FC ", q_FC[1])
    println("q_DA ", value.(q_DA[1]))
    println("q_15 ", value.(q_15[1,1]))
    println("mean R ", Statistics.mean(value.(R[1,1])))
    println("STD R ", Statistics.std(value.(R[1,1])))
    println("CVaR ", Statistics.mean(value.(CVaR[:])))
    
end;

In [37]:
wind_opt()

β: 0.0
p_DA 16.81
p_15 16.85
p_ID3 20.28
q_FC 0.0
q_DA 0.0
q_15 0.0
mean R 0.0
STD R NaN
CVaR 0.0


In [38]:
wind_opt(10)

β: 10
p_DA 16.81
p_15 16.85
p_ID3 20.28
q_FC 0.0
q_DA 0.0
q_15 0.0
mean R 0.0
STD R NaN
CVaR 0.0


In [ ]:
### OLD: Save in Dict
par = Dict(
    ### Sets (or rather, set cardinalities)
    # Number scenarios
    :Ω => 100,
    # Number time periods
    :T => 2,
    
    ### Scalars
    # Factors to FC that limit DA bids
    :FC_DA_fac_lo => 0.85,
    :FC_DA_fac_up => 1.15,
    
    ### Vectors ###
    ## Length T
    # Prices in DA, 15 and ID
    :p_DA  => data["time_series"][1:param[:T],"p_da"],
    :p_15  => data["time_series"][1:param[:T],"p_15"],
    :p_ID3 => data["time_series"][1:param[:T],"p_id3"],
    # Power ForeCasted and real
    :q_FC => ones(T)
    :q_r => ones(T)
    ## Length Ω
    # FC error sign; 1st stage decission
    :F_sgn => append!(-1 * ones(size(data["scenarios"].neg)), ones(size(data["scenarios"].pos))),
    # Relative FC errors
    :Δq_FC => append!(data["scenarios"].neg, data["scenarios"].pos),
    # Scenarios probablilities
    :pi => 1/param[:Ω] * ones(param[:Ω]),
    
    
)

In [ ]:
### OLD: Stages kriegen einzelne Nummerierungen
par = Dict(
    ### Sets (or rather, set cardinalities)
    # Number scenarios first stage
    :Ω => 2,
    # Number scenarios sec stage
    :Φ => 50,
    # Number time periods
    :T => 2,
    
    ### Scalars
    # Factors to FC that limit DA bids
    :FC_DA_fac_lo => 0.85,
    :FC_DA_fac_up => 1.15,
    
    ### Vectors
    # Prices in DA, 15 and ID
    :p_DA  => data["time_series"][1:param[:T],"p_da"],
    :p_15  => data["time_series"][1:param[:T],"p_15"],
    :p_ID3 => data["time_series"][1:param[:T],"p_id3"],
    # Scenarios probablilities
    :pi_ω => 1/param[:Ω] * ones(param[:Ω]),
    :pi_ϕ => 1/param[:Φ] * ones(param[:Φ]),
    # 1st stage decission: Forecast sign
    :FC_sgn => [-1, +1],
    
    
);